In [159]:
# determine the optimal pre-match forecast, given elo_diff and s_elo_diff
# step 2: calculate glicko and s_glicko for each player and do the same

In [165]:
# can test this on our subset of 10,000 matches as well as all matches in the database:
df = pd.read_csv('../my_data/elo_pbp_with_surface_8_22.csv')
del df['Unnamed: 0']
df.head(2)

,tourney_id,tourney_name,tourney_date,p0_name,p1_name,p0_elo,p1_elo,elo_diff,p0_s_elo,p1_s_elo,...,p0_r_pct_JS,p1_52_rwon,p1_52_rpt,p1_r_pct,p1_r_pct_JS,tourney_stats,best_of,score,pbp,winner
0,2010-375,Montpellier,2010-10-25,Romain Jouan,Taylor Dent,1519.128216,1720.828131,-201.699915,1517.563000,1669.753884,...,0.343856,750.0,2182.0,0.343721,0.344525,0.66189,3,6-7 6-3 6-4,SSSS;SSSS;SSSS;SSSS;SSSS;SRRSSS;RRSSRSSS;SSSS;...,1
1,2010-375,Montpellier,2010-10-25,Julian Reister,Richard Gasquet,1574.319533,1948.840391,-374.520858,1472.470266,1839.154165,...,0.353614,1624.0,4348.0,0.373505,0.372607,0.66189,3,6-7 6-3 6-3,SSSS;SSSS;SSSS;SSSS;SSSS;SSSS;SSSS;SSSS;SSRRSR...,1


In [166]:
print 'elo baseline: ',  sum((df['elo_diff']<0) == df['winner'])/float(len(df))
print 'surface elo baseline: ', sum((df['s_elo_diff']<0) == df['winner'])/float(len(df))

current = (0,0)
for weight in np.arange(10)*.1:
    accuracy = sum(((1-weight)*df['elo_diff']+weight*df['s_elo_diff']<0) == df['winner'])/float(len(df))
    if accuracy > current[1]:
        current=weight,accuracy
print 'best s_elo weight: ',current[0],'accuracy =',current[1]
df['elo_diff_weighted'] = (1-current[1])*df['elo_diff']+current[1]*df['s_elo_diff']

elo baseline:  0.696569178853
surface elo baseline:  0.691038620172
best s_elo weight:  0.3 accuracy = 0.702099737533


In [220]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,accuracy_score

kfold = KFold(n_splits=5,shuffle=True)
scores = [],[]
cols = ['elo_diff','s_elo_diff']
for train_ind,test_ind in kfold.split(df):
    lm = linear_model.LogisticRegression(fit_intercept = True)
    train_df,test_df = df.loc[train_ind],df.loc[test_ind]
    lm.fit(sub_df[cols].values.reshape([len(sub_df),len(cols)]),sub_df['winner'])
    y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
    y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
    accuracy = accuracy_score(test_df['winner'],y_preds)
    loss = log_loss(test_df['winner'],y_probs,labels=[0,1])
    scores[0].append(accuracy);scores[1].append(loss)
print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
print 'accuracy: ', np.mean(scores[0])
print 'loss: ', np.mean(scores[1])

% s_elo used in lm fit:  0.43309008408
accuracy:  0.722475247525
loss:  0.538804439863


In [222]:
lm.coef_

array([[-0.00291231, -0.00222485]])

In [211]:
# logistic regression on z scores of our probabilities AND elo_diff/s_elo_diff
from scipy.stats import norm
df_all = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
del df_all['Unnamed: 0']
for col in ['sackmann_prob','klaassen_prob','klaassen_prob_a_400']:
    df_all[col.replace('prob','z')] = norm.ppf(df_all[col])
df_all.loc[df_all['sackmann_z']>10,'sackmann_z'] = 10
df_all.loc[df_all['klaassen_z']>10,'klaassen_z'] = 10
df_all.loc[df_all['klaassen_z_a_400']>10,'klaassen_z_a_400'] = 10

In [213]:
max(df_all['klaassen_z_a_400'])

10.0

In [217]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,accuracy_score

# also will want to try ensemble methods at some point...
kfold = KFold(n_splits=5,shuffle=True)
scores = [],[]
cols = ['elo_diff','s_elo_diff','klaassen_z_a_400']
for train_ind,test_ind in kfold.split(df_all):
    lm = linear_model.LogisticRegression(fit_intercept = True)
    train_df,test_df = df_all.loc[train_ind],df_all.loc[test_ind]
    lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
    y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
    y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
    accuracy = accuracy_score(test_df['winner'],y_preds)
    loss = log_loss(test_df['winner'],y_probs,labels=[0,1])
    scores[0].append(accuracy);scores[1].append(loss)
#print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
print 'coefficients: ', lm.coef_[0]
print 'accuracy: ', np.mean(scores[0])
print 'loss: ', np.mean(scores[1])

coefficients:  [ -5.45725783e-04   1.73519587e-03   1.71256863e+00]
accuracy:  0.77803661972
loss:  0.456071683438


In [ ]:
# perform same analysis across entire match dataset, over the decades

In [161]:
# one-time fix to transfer over the elo_diff's
# import pandas as pd
# import numpy as np

# df_og = pd.read_csv('../my_data/elo_pbp_with_surface_8_22.csv')
# del df_og['Unnamed: 0']
# df = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
# del df['Unnamed: 0']

# elo_dict = dict(zip(range(len(df_og)),df_og['elo_diff']))
# s_elo_dict = dict(zip(range(len(df_og)),df_og['s_elo_diff']))

# df['elo_diff'] = [elo_dict[m_id] for m_id in df['match_id']]
# df['s_elo_diff'] = [s_elo_dict[m_id] for m_id in df['match_id']]
# df.to_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')